# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [1]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [13]:
TRAIN_START_DATE = '2020-01-01'
TRADE_END_DATE = '2020-01-31'
aapl_df_yf = yf.download(tickers = "aapl", start=TRAIN_START_DATE, end=TRADE_END_DATE)
# aapl_df_yf = yf.download(tickers = "msft", start=TRAIN_START_DATE, end=TRADE_END_DATE)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [6]:
aapl_df_yf.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [34]:
aapl_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRAIN_END_DATE,
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


ValueError: no data is fetched.

In [7]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,73.449394,135480400,aapl,3
1,2020-01-03,74.287498,75.144997,74.125000,72.735313,146322800,aapl,4
2,2020-01-06,73.447502,74.989998,73.187500,73.314888,118387200,aapl,0
3,2020-01-07,74.959999,75.224998,74.370003,72.970078,108872000,aapl,1
4,2020-01-08,74.290001,76.110001,74.290001,74.143906,132079200,aapl,2


## Data for the chosen tickers

In [8]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [9]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [10]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.758536,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,43.832630,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.365304,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941090,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,31.579340,7117200,CAT,4


## 数据无法从yahoo下载下来

只能造一个假数据。

In [38]:
df1 = pd.read_csv('../examples/data/demo_train_data.csv')
df2 = pd.read_csv('../examples/data/demo_trade_data.csv')

df_raw = pd.concat([df1, df2])
# df_raw['date'] = pd.to_datetime(df_raw['date'])
df_raw = df_raw.sort_values(by='date')
df_raw['day'] = df_raw['day'].astype(int)

selected_cols = ['date',  'open', 'high', 'low', 'close', 'volume', 'tic', 'day']
df_raw = df_raw[selected_cols]
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.758536,746015200.0,AAPL,4
28,2009-01-02,55.980000,57.509998,55.779999,41.618889,16054800.0,WMT,4
27,2009-01-02,24.650000,25.670000,24.610001,17.748915,5266700.0,WBA,4
26,2009-01-02,32.000774,32.601021,31.466175,16.240591,14848985.0,VZ,4
25,2009-01-02,13.230000,13.427500,13.060000,12.079281,13199200.0,V,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [39]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=False, # 原来是true
                     use_turbulence=False,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


In [40]:
processed.head() # 这里没有vix和turbulence，只是为了跑通

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2009-01-02,3.067143,3.251429,3.041429,2.758536,746015200.0,AAPL,4,0.0,2.98139,2.652102,100.0,66.666667,100.0,2.758536,2.758536
3228,2009-01-02,58.590000,59.080002,57.750000,44.219181,6547900.0,AMGN,4,0.0,2.98139,2.652102,100.0,66.666667,100.0,44.219181,44.219181
6456,2009-01-02,18.570000,19.520000,18.400000,15.365305,10955700.0,AXP,4,0.0,2.98139,2.652102,100.0,66.666667,100.0,15.365305,15.365305
9684,2009-01-02,42.799999,45.560001,42.779999,33.941093,7010200.0,BA,4,0.0,2.98139,2.652102,100.0,66.666667,100.0,33.941093,33.941093
12912,2009-01-02,44.910000,46.980000,44.709999,31.579342,7117200.0,CAT,4,0.0,2.98139,2.652102,100.0,66.666667,100.0,31.579342,31.579342


In [ ]:
# # 本来是要 use_vix 和 turbulence 的
# fe = FeatureEngineer(use_technical_indicator=True,
#                      tech_indicator_list = INDICATORS,
#                      use_vix=True,
#                      use_turbulence=True,
#                      user_defined_feature = False)

# processed = fe.preprocess_data(df_raw)

In [41]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [42]:
processed_full.head() # 没有vis和turbulence了

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758536,746015200.0,4.0,0.0,2.98139,2.652102,100.0,66.666667,100.0,2.758536,2.758536
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,44.219181,6547900.0,4.0,0.0,2.98139,2.652102,100.0,66.666667,100.0,44.219181,44.219181
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365305,10955700.0,4.0,0.0,2.98139,2.652102,100.0,66.666667,100.0,15.365305,15.365305
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941093,7010200.0,4.0,0.0,2.98139,2.652102,100.0,66.666667,100.0,33.941093,33.941093
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579342,7117200.0,4.0,0.0,2.98139,2.652102,100.0,66.666667,100.0,31.579342,31.579342


In [16]:
processed_full.head() # 这是原来的代码的形式

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758536,746015200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,2.758536,2.758536,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832630,6547900.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,43.832630,43.832630,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365304,10955700.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,15.365304,15.365304,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941090,7010200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,33.941090,33.941090,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,31.579340,31.579340,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [47]:
# 感觉像是date不是datetime类型，就无法split
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

4234
0


In [46]:
TRADE_END_DATE

'2020-01-31'

In [45]:
processed_full.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
135773,2021-10-27,UNH,454.640015,460.440002,453.480011,446.290222,3520400.0,2.0,11.476812,456.799667,366.375928,65.085944,174.303055,51.571330,408.568605,408.856444
135774,2021-10-27,V,224.750000,224.750000,215.660004,213.365738,22958100.0,2.0,0.013879,234.385357,216.105305,41.737198,-153.763693,30.337021,224.350456,226.572504
135775,2021-10-27,VZ,53.169998,53.200001,52.470001,49.126247,15007400.0,2.0,-0.228182,50.933580,47.760363,44.604890,-50.042829,8.504850,49.590407,50.228507
135776,2021-10-27,WBA,48.450001,48.459999,47.090000,44.605289,5652000.0,2.0,-0.015552,47.395886,43.400294,46.167786,-62.766485,5.045608,45.612187,45.950003
135777,2021-10-27,WMT,148.960007,149.020004,147.389999,144.664368,4855200.0,2.0,1.531603,147.995390,130.180237,57.061087,124.768634,30.006323,139.379048,142.085952


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [18]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')